# Second Normal Form (2NF)

A table is in 2NF if it is in 1NF and all non-key columns are fully functionally dependent on the primary key.

By fully functionally dependent, we mean that if a non-key column is dependent on only part of a composite primary key, then the table is not in 2NF.

## Table Violating 2NF

| OrderID | ProductID | ProductName |
|---------|-----------|-------------|
| 1       | 101       | Laptop      |
| 1       | 102       | Mouse       |
| 2       | 101       | Laptop      |

**Why it violates 2NF:** The "ProductName" column depends only on "ProductID" and not on the composite primary key (OrderID, ProductID).

In [1]:
# Import SQLite library
import sqlite3

# Create an in-memory SQLite database
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

In [2]:
# Create a table that violates 2NF
cursor.execute('''
CREATE TABLE Orders (
    OrderID INTEGER,
    ProductID INTEGER,
    ProductName TEXT,
    PRIMARY KEY (OrderID, ProductID)
)''')

# Insert data
cursor.executemany('INSERT INTO Orders (OrderID, ProductID, ProductName) VALUES (?, ?, ?)', [
    (1, 101, 'Laptop'),
    (1, 102, 'Mouse'),
    (2, 101, 'Laptop')
])

# Query the table
cursor.execute('SELECT * FROM Orders')
for row in cursor.fetchall():
    print(row)

(1, 101, 'Laptop')
(1, 102, 'Mouse')
(2, 101, 'Laptop')


## Converting to 2NF

To achieve 2NF, we separate the `ProductName` into a new table.

In [3]:
# Create tables in 2NF
cursor.execute('''
CREATE TABLE Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT
)''')

cursor.execute('''
CREATE TABLE OrderDetails (
    OrderID INTEGER,
    ProductID INTEGER,
    PRIMARY KEY (OrderID, ProductID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
)''')

# Insert data into 2NF tables
cursor.executemany('INSERT INTO Products (ProductID, ProductName) VALUES (?, ?)', [
    (101, 'Laptop'),
    (102, 'Mouse')
])

cursor.executemany('INSERT INTO OrderDetails (OrderID, ProductID) VALUES (?, ?)', [
    (1, 101),
    (1, 102),
    (2, 101)
])

# Query the 2NF tables
print('Products Table:')
cursor.execute('SELECT * FROM Products')
for row in cursor.fetchall():
    print(row)

print('\nOrderDetails Table:')
cursor.execute('SELECT * FROM OrderDetails')
for row in cursor.fetchall():
    print(row)

Products Table:
(101, 'Laptop')
(102, 'Mouse')

OrderDetails Table:
(1, 101)
(1, 102)
(2, 101)
